<a href="https://colab.research.google.com/github/YounSooKimTech/Joy_Profile_Photos/blob/main/Cornell_Meta_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta Data

In [1]:
df_department = []
df_name =[]
df_url = []


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

departments = ["accounting", "applied-economics-and-policy", "finance", "management-and-organizations",
               "marketing-and-management-communication","operations-technology-and-information-management",
               "services-management", "strategy-and-business-economics"]

for a_department in departments:
  url = f"https://business.cornell.edu/faculty-research/areas/{a_department}/"
  response = requests.get(url)
  # print(url, response)

  soup = BeautifulSoup(response.content, 'html.parser')

  faculty_tags = soup.find_all("ul", class_="faculty--columns")
  for faculty_tag in faculty_tags:
    li_tags = faculty_tag.find_all("li")
    for li_tag in li_tags:
      name = li_tag.get_text().strip()
      href_value = li_tag.find('a')['href']
      response = requests.get(href_value)
      print(a_department, name, href_value, response)

      df_department.append(a_department)
      df_name.append(name)
      df_url.append(href_value)


accounting Sanjeev Bhojraj https://business.cornell.edu/faculty-research/faculty/sb235/ <Response [200]>
accounting Robert J. Bloomfield https://business.cornell.edu/faculty-research/faculty/rjb9/ <Response [200]>
accounting Thomas Godwin https://business.cornell.edu/faculty-research/faculty/rtg77/ <Response [200]>
accounting Nicholas Guest https://business.cornell.edu/faculty-research/faculty/nmg75/ <Response [200]>
accounting Ryan Guggenmos https://business.cornell.edu/faculty-research/faculty/rdg222/ <Response [200]>
accounting Eric Lewis https://business.cornell.edu/faculty-research/faculty/eel33/ <Response [200]>
accounting Robert Libby https://business.cornell.edu/faculty-research/faculty/rl54/ <Response [200]>
accounting Yao Lu https://business.cornell.edu/faculty-research/faculty/yl2269/ <Response [200]>
accounting Mary MacAusland https://business.cornell.edu/faculty-research/faculty/mm2549/ <Response [200]>
accounting Asis Martinez-Jerez https://business.cornell.edu/faculty-re

In [3]:
import pandas as pd

df_meta = pd.DataFrame({"Department":df_department,
                        "Name":df_name,
                        "URL":df_url})

df_meta

,Department,Name,URL
0,accounting,Sanjeev Bhojraj,https://business.cornell.edu/faculty-research/...
1,accounting,Robert J. Bloomfield,https://business.cornell.edu/faculty-research/...
2,accounting,Thomas Godwin,https://business.cornell.edu/faculty-research/...
3,accounting,Nicholas Guest,https://business.cornell.edu/faculty-research/...
4,accounting,Ryan Guggenmos,https://business.cornell.edu/faculty-research/...
...,...,...,...
374,strategy-and-business-economics,Elena Iankova,https://business.cornell.edu/faculty-research/...
375,strategy-and-business-economics,Vrinda Kadiyali,https://business.cornell.edu/faculty-research/...
376,strategy-and-business-economics,Nicholas Sanders,https://business.cornell.edu/faculty-research/...
377,strategy-and-business-economics,Deborah Streeter,https://business.cornell.edu/faculty-research/...


## Title and Biography

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

df_title = []
df_bio = []


for url in df_meta["URL"]:
   response = requests.get(url)
   soup = BeautifulSoup(response.text, 'html.parser')

  # title
   div_tag = soup.find("div", class_="faculty-profile__content")
   ul_tag = div_tag.find("ul", class_="faculty-profile__list")  if div_tag else None
   title = ul_tag.get_text().strip()  if ul_tag else ""

   # biography
   bio_element = soup.find("p", class_="faculty-profile__bio")
   biography = bio_element.get_text().strip() if bio_element else ""

   df_title.append(title)
   df_bio.append(biography)

   #print(url, title, biography)


In [5]:
df_meta["Title"] = df_title
df_meta["Biography"] = df_bio

In [6]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Department  379 non-null    object
 1   Name        379 non-null    object
 2   URL         379 non-null    object
 3   Title       379 non-null    object
 4   Biography   379 non-null    object
dtypes: object(5)
memory usage: 14.9+ KB


In [59]:
df_meta_prof = df_meta[~df_meta["Title"].str.contains("Visiting|Lecturer|Adjunct|Clinical|Postdoctoral|Practice|Extension")]

df_meta_prof = df_meta_prof[df_meta_prof["Title"] != "Professor of Practice"]

df_meta_prof = df_meta_prof[df_meta_prof["Title"] != "Research Associate"]
df_meta_prof = df_meta_prof[df_meta_prof["Title"] != "Senior Research Associate"]

df_meta_prof = df_meta_prof[df_meta_prof["Title"] != "Extension Associate"]
df_meta_prof = df_meta_prof[df_meta_prof["Title"] != "Senior Extension Associate Emeritus"]

# Yuchen Wu <- No page exist
df_meta_prof = df_meta_prof[df_meta_prof["Title"] != ""]

In [60]:
df_meta_prof.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203 entries, 0 to 377
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Department  203 non-null    object
 1   Name        203 non-null    object
 2   URL         203 non-null    object
 3   Title       203 non-null    object
 4   Biography   203 non-null    object
dtypes: object(5)
memory usage: 9.5+ KB


## Rank based on title

In [61]:

import numpy as np

conditions = [
    df_meta_prof["Title"].str.contains("Emeritus"),
    df_meta_prof["Title"].str.contains("Emerita"),
    df_meta_prof["Title"].str.contains("Assistant"),
    df_meta_prof["Title"].str.contains("Associate")
]

choices = ["Emeritus", "Emeritus", "Assistant", "Associate"]

df_meta_prof["Rank"] = np.select(conditions, choices, default="Full")

df_meta_prof["University"] = "Cornell"

# Sex based on Biography Pronouns

In [62]:

import spacy
nlp = spacy.load('en_core_web_sm')

pronouns_list = []

for text in df_meta_prof['Biography']:
    doc = nlp(text)
    pronouns = [token.text.lower() for token in doc if token.pos_ == 'PRON' and token.text.lower() in ['he', 'she', 'her', 'his']]
    pronouns_list.append(pronouns)

df_meta_prof['Pronouns'] = pronouns_list

def determine_sex(pronouns):
    if 'he' in pronouns or 'his' in pronouns:
        return 'Male'
    elif 'she' in pronouns or 'her' in pronouns:
        return 'Female'
    else:
        return 'Unknown'

df_meta_prof['Sex'] = df_meta_prof['Pronouns'].apply(determine_sex)

df_meta_prof[["Pronouns", "Sex"]]

,Pronouns,Sex
0,"[he, his, he, he, he, he, his]",Male
1,[he],Male
3,"[his, he]",Male
4,"[his, he, his, he, his, his, his, he, his]",Male
6,"[his, his, his, his, his, he, he, he]",Male
...,...,...
369,"[she, her, her, her, her]",Female
371,"[she, her, her, she, her, her]",Female
372,"[his, he, his, he, his, his, he]",Male
375,"[her, she, she, she, she]",Female


In [63]:
# Missing Sex
# https://www.cornell.edu/video/asis-martinez-jerez-on-servant-leadership
# https://en.wikipedia.org/wiki/Kaushik_Basu
# https://www.cornell.edu/video/dragana-cvijanovic-realistic-market-shocks
# https://business.cornell.edu/hub/2022/12/05/meet-sean-flynn/
# https://heep.hks.harvard.edu/people/todd-gerarden
# https://www.human.cornell.edu/people/jfh246
# https://business.cornell.edu/hub/2023/07/10/meet-our-new-faculty-lauri-kytomaa/
# https://heep.hks.harvard.edu/people/cynthia-lin

df_meta_prof.loc[df_meta_prof["Name"] == "Asis Martinez-Jerez", "Sex"] = "Male"
df_meta_prof.loc[df_meta_prof["Name"] == "Kaushik Basu", "Sex"] = "Male"

df_meta_prof.loc[df_meta_prof["Name"] == "Dragana Cvijanovic", "Sex"] = "Female"
df_meta_prof.loc[df_meta_prof["Name"] == "Sean Flynn", "Sex"] = "Male"

df_meta_prof.loc[df_meta_prof["Name"] == "Todd Gerarden", "Sex"] = "Male"
df_meta_prof.loc[df_meta_prof["Name"] == "John Hoddinott", "Sex"] = "Male"

df_meta_prof.loc[df_meta_prof["Name"] == "Lauri Kytomaa", "Sex"] = "Male"
df_meta_prof.loc[df_meta_prof["Name"] == "C.-Y. Cynthia Lin Lawell", "Sex"] = "Female"

In [64]:
# https://infosci.cornell.edu/content/michuda
# https://equitablegrowth.org/people/ivan-rudik/
# https://business.cornell.edu/hub/2023/07/31/meet-our-new-faculty-heather-schofield/
# https://www.canr.msu.edu/people/wolf
# https://business.cornell.edu/hub/2022/12/09/meet-our-new-faculty-kelly-posenau/
# https://business.cornell.edu/hub/2022/09/28/meet-our-new-faculty-mao-ye-phd-11/

df_meta_prof.loc[df_meta_prof["Name"] == "Aleksandr Michuda", "Sex"] = "Male"
df_meta_prof.loc[df_meta_prof["Name"] == "Ivan Rudik (on leave)", "Sex"] = "Male"

df_meta_prof.loc[df_meta_prof["Name"] == "Heather Schofield", "Sex"] = "Female"
df_meta_prof.loc[df_meta_prof["Name"] == "Christopher A. Wolf", "Sex"] = "Male"

df_meta_prof.loc[df_meta_prof["Name"] == "Kelly Posenau", "Sex"] = "Female"
df_meta_prof.loc[df_meta_prof["Name"] == "Mao Ye", "Sex"] = "Male"

In [67]:
# https://business.cornell.edu/hub/2023/07/28/meet-our-new-faculty-jason-greenberg/
# https://business.cornell.edu/hub/2023/08/10/meet-our-new-faculty-wyatt-lee/
# https://ler.la.psu.edu/news/faculty-feature-q-a-with-michael-maffie/
# https://www.cornell.edu/video/elizabeth-mcclean-external-pressure-to-pay-women-equitably
# https://lauraniemi.com/
# https://www.cornell.edu/video/simone-tang-on-the-humanization-of-organizations

df_meta_prof.loc[df_meta_prof["Name"] == "Jason Greenberg", "Sex"] = "Male"
df_meta_prof.loc[df_meta_prof["Name"] == "Wyatt Lee", "Sex"] = "Male"

df_meta_prof.loc[df_meta_prof["Name"] == "Michael D. Maffie", "Sex"] = "Male"
df_meta_prof.loc[df_meta_prof["Name"] == "Elizabeth McClean", "Sex"] = "Female"

df_meta_prof.loc[df_meta_prof["Name"] == "Laura Niemi", "Sex"] = "Female"
df_meta_prof.loc[df_meta_prof["Name"] == "Simone Tang", "Sex"] = "Female"


In [69]:
# https://tech.cornell.edu/people/omid-rafieian/
# https://business.cornell.edu/hub/2023/07/17/meet-our-new-faculty-yichun-hu/
# https://business.cornell.edu/hub/2022/12/13/meet-our-new-faculty-nur-kaynar/
# https://business.cornell.edu/hub/2022/12/16/meet-our-new-faculty-collin-raymond/ <- He & SHe
# https://www.linkedin.com/in/danielascur/

df_meta_prof.loc[df_meta_prof["Name"] == "Omid Rafieian", "Sex"] = "Male"
df_meta_prof.loc[df_meta_prof["Name"] == "Yichun Hu", "Sex"] = "Female"

df_meta_prof.loc[df_meta_prof["Name"] == "Nur Kaynar", "Sex"] = "Female"
df_meta_prof.loc[df_meta_prof["Name"] == "Collin Raymond", "Sex"] = "Male"

df_meta_prof.loc[df_meta_prof["Name"] == "Daniela Scur", "Sex"] = "Female"

In [70]:
df_meta_prof[df_meta_prof["Sex"] == "Unknown"]

,Department,Name,URL,Title,Biography,Rank,University,Pronouns,Sex
113,applied-economics-and-policy,William G. Tomek,https://business.cornell.edu/faculty-research/...,Professor Emeritus,William Tomek is Professor Emeritus at the Dys...,Emeritus,Cornell,[],Unknown
132,finance,Justin Murfin,https://business.cornell.edu/faculty-research/...,Associate Professor,,Associate,Cornell,[],Unknown
355,strategy-and-business-economics,Adam Dearing,https://business.cornell.edu/faculty-research/...,Assistant Professor,,Assistant,Cornell,[],Unknown


In [72]:
df_meta_prof_no_unknown = df_meta_prof[df_meta_prof["Sex"] != "Unknown"]

In [76]:
df_meta_prof_no_unknown.to_csv("Cornell_Prof_Meta.csv", index=False)

# Cornell Photo

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# management-organization error

departments = ["accounting", "finance", "marketing", "operations-technology-information-management", "economics"]

df_names = []
df_departments = []
df_links = []

for department in departments:
  url = f"https://www.johnson.cornell.edu/programs/phd-program/{department}/"
  response = requests.get(url)
  # print(url, response)
  soup = BeautifulSoup(response.content, 'html.parser')

  first_tag = soup.find("div", class_="g-row g-row--center g-row--col-3--min-full")
  p_tags = first_tag.find_all('p')

  for p_tag in p_tags:
    # Find the <a> tag within each <p> tag
    prof_link = p_tag.find('a')
    name = prof_link.get_text().split("\n")[0]
    href = prof_link["href"]
    response = requests.get(href)

    df_names.append(name)
    df_links.append(href)
    df_departments.append(department)

In [ ]:
import pandas as pd

df = pd.DataFrame({"Name":df_names,
                   "Department":df_departments,
                   "URL":df_links})

#df["Filename"] = df["URL"].apply(lambda x: x.split("/")[-2])

df

,Name,Department,URL,Filename
0,Sanjeev Bhojraj,accounting,https://www.johnson.cornell.edu/faculty-resear...,sb235
1,Robert J. Bloomfield,accounting,https://www.johnson.cornell.edu/faculty-resear...,rjb9
2,Nicholas Matthew Guest,accounting,https://www.johnson.cornell.edu/faculty-resear...,nmg75
3,Ryan Guggenmos,accounting,https://www.johnson.cornell.edu/faculty-resear...,rdg222
4,Robert Libby,accounting,https://www.johnson.cornell.edu/faculty-resear...,rl54
...,...,...,...,...
82,Marcel Preuss,economics,https://www.johnson.cornell.edu/faculty-resear...,mp2222
83,Imke Reimers,economics,https://business.cornell.edu/faculty-research/...,icr9
84,Daniela Scur,economics,https://dyson.cornell.edu/faculty-research/fac...,ds2338
85,Michael Waldman,economics,https://www.johnson.cornell.edu/faculty-resear...,mw46


In [ ]:
urls = list(df["URL"])
titles = []
biographies = []

for url in df["URL"]:
    if url == "https://www.johnson.cornell.edu/faculty-research/faculty/lwr2/":
        title = ""
        biography = ""
    else:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Titles
        title_element = soup.find("ul", class_="faculty-profile__list")
        title = title_element.get_text().strip() if title_element else None

        # Biography
        bio_element = soup.find("p", class_="faculty-profile__bio")
        biography = bio_element.get_text().strip() if bio_element else ""

    titles.append(title)
    biographies.append(biography)

# Add the titles and biographies lists as new columns to the DataFrame
df["Title"] = titles
df["Biography"] = biographies

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

pronouns_list = []

for text in df['Biography']:
    doc = nlp(text)
    pronouns = [token.text.lower() for token in doc if token.pos_ == 'PRON' and token.text.lower() in ['he', 'she', 'her', 'his']]
    pronouns_list.append(pronouns)

df['Pronouns'] = pronouns_list

def determine_sex(pronouns):
    if 'he' in pronouns or 'his' in pronouns:
        return 'Male'
    elif 'she' in pronouns or 'her' in pronouns:
        return 'Female'
    else:
        return 'Unknown'

df['Sex'] = df['Pronouns'].apply(determine_sex)

df[["Pronouns", "Sex"]]

,Pronouns,Sex
0,"[he, his, he, he, he, he, his]",Male
1,[he],Male
2,"[his, he]",Male
3,"[his, he, his, he, his, his, his, he, his]",Male
4,"[his, his, his, his, his, he, he, he]",Male
...,...,...
82,"[his, he, he, he, his, he, his, his, he]",Male
83,"[she, her, she, she]",Female
84,[],Unknown
85,"[he, his, his, he, his, his, he, he, he, he, h...",Male


In [ ]:
df[df["Sex"] == "Unknown"]

,Name,Department,URL,Filename,Title,Biography,Pronouns,Sex
6,Asis Martinez-Jerez,accounting,https://sha.cornell.edu/faculty-research/facul...,fm447,Associate Professor,,[],Unknown
30,Justin Murfin,finance,https://dyson.cornell.edu/faculty-research/fac...,jm2692,Associate Professor,,[],Unknown
50,Omid Rafieian,marketing,https://www.johnson.cornell.edu/faculty-resear...,or83,Assistant Professor\nDemir Sabanci Faculty Fel...,,[],Unknown
66,Yichun Hu,operations-technology-information-management,https://business.cornell.edu/faculty-research/...,yh767,Assistant Professor,,[],Unknown
67,Nur Kaynar,operations-technology-information-management,https://www.johnson.cornell.edu/faculty-resear...,sk2739,Assistant Professor,,[],Unknown
69,Lawrence W. Robinson,operations-technology-information-management,https://www.johnson.cornell.edu/faculty-resear...,lwr2,,,[],Unknown
74,Adam Dearing,economics,https://www.johnson.cornell.edu/faculty-resear...,aed237,Assistant Professor,,[],Unknown
84,Daniela Scur,economics,https://dyson.cornell.edu/faculty-research/fac...,ds2338,Assistant Professor,I study organizations and how organizational p...,[],Unknown


In [ ]:
urls = list(df["URL"])
titles =[]

for url in urls:
    if url== "https://www.johnson.cornell.edu/faculty-research/faculty/lwr2/":
      title = None
    else:
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      title = soup.find("ul", class_="faculty-profile__list").get_text().strip()

      # Biograhphy
      prof_text = soup.find("p", class_ = "faculty-profile__bio").get_text().strip()
      print(prof_text)

    titles.append(title)


Professor Sanjeev Bhojraj is the Alumni Professor in Asset Management and the Faculty Co-Director of the Parker Center for Investment Research. He has been a part of the faculty of the Cornell Johnson Graduate School of Management since 1999 and served as the sole designated manager of the Cayuga Fund from 2005-2014. His research interests are in the areas of behavioral finance, corporate governance, discretionary disclosure of information by firms, and international accounting and valuation. He has taught courses in applied portfolio management, financial analysis, and financial accounting. Professor Bhojraj is an award-winning instructor and has received several teaching awards from the MBA and Executive MBA classes, most recently the 2023 Apple Award for Teaching Excellence. He was featured as Cornell's highest rated professor in The Wall Street Journal's "Special Report" on executive MBA programs. He also received outstanding faculty recognition in BusinessWeek's "Guide to The Best

AttributeError: ignored

In [ ]:
df["Title"] = titles


df.head()

,Name,Department,URL,Filename,Title
0,Sanjeev Bhojraj,accounting,https://www.johnson.cornell.edu/faculty-resear...,sb235,Alumni Professor in Asset Management\nFaculty ...
1,Robert J. Bloomfield,accounting,https://www.johnson.cornell.edu/faculty-resear...,rjb9,Nicholas H. Noyes Professor of Management\nAre...
2,Nicholas Matthew Guest,accounting,https://www.johnson.cornell.edu/faculty-resear...,nmg75,Assistant Professor
3,Ryan Guggenmos,accounting,https://www.johnson.cornell.edu/faculty-resear...,rdg222,Assistant Professor
4,Robert Libby,accounting,https://www.johnson.cornell.edu/faculty-resear...,rl54,David A. Thomas Professor of Management


In [ ]:
titles = []

for index, row in df.iterrows():
    filename = f'{row["Filename"]}.jpg'
    url = row["URL"]

    if url== "https://www.johnson.cornell.edu/faculty-research/faculty/lwr2/":
      title = None

   # scrap the titles
    else:
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')

      title = soup.find("ul", class_="faculty-profile__list").get_text().strip()
      titles.append(title)


KeyboardInterrupt: ignored

In [ ]:
urls = list(df["URL"])
titles =[]

for url in urls:
    if url== "https://www.johnson.cornell.edu/faculty-research/faculty/lwr2/":
      title = None
    else:
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      title = soup.find("ul", class_="faculty-profile__list").get_text().strip()
      titles.append(title)

      filename = f'url.split("/")[-2].jpg'
      print(filename)

      figure_tag = soup.find('figure')
      img_tag = soup.find("img")
      img_src = img_tag.find()

    #print(url)
    #print(figure_tag)


sb235
rjb9
nmg75
rdg222
rl54
yl2269
fm447
mwn2
mtp58
kmr52
ss3647
yw2652
pey4
xz687
jma369
wbb1
mdb327
vlb23
mnc35
lc29
sac20
lc898
aac256
mg2457
mh375
raj15
gak56
chl62
ym355
pm388
jm2692
dtn4
mo19
gs25
sey8
tb558
kb746
faculty
csd5
gwf25
agf52
eng26
sg248
sh2596
vk24
rjk34
kal276
jl2545
wml3
yp34
or83
sbs78
dms43
mkt27
smv64
krw67
ncy6
aea68
cka9
eb733
lc785
jpc355
yc2268
amd365
vg77
kg488
yh767
sk2739
mq56
vws8
gmt1
jz2293
rz383
aed237
yc2535
cmf257
oh33
jpj25
tj268
ael24
bl693
mp2222
icr9
ds2338
mw46
sew276
